# 使用 Milvus 和 DeepSeek 构建 RAG

DeepSeek 帮助开发者使用高性能语言模型构建和扩展 AI 应用。它提供高效的推理、灵活的 API 以及先进的专家混合 (MoE) 架构，用于强大的推理和检索任务。

在本教程中，我们将展示如何使用 Milvus 和 DeepSeek 构建一个检索增强生成 (RAG) 管道。

## 准备工作

### 依赖与环境

In [ ]:
!pip install "pymilvus[model]==2.5.10" openai==1.82.0 requests==2.32.3 tqdm==4.67.1 torch==2.7.0

---

In [1]:
import os

# 从环境变量获取 DeepSeek API Key
api_key = os.getenv("DEEPSEEK_API_KEY")

In [2]:
print(api_key)

sk-3a61e32281444e2a82a94d8e8762a9f7


### 准备数据

我们使用 Milvus 文档 2.4.x 中的 FAQ 页面作为我们 RAG 中的私有知识库，这是一个简单 RAG 管道的良好数据源。

下载 zip 文件并将文档解压到 `milvus_docs` 文件夹。

**建议在命令行执行下面命令**

In [3]:
#!wget https://github.com/milvus-io/milvus-docs/releases/download/v2.4.6-preview/milvus_docs_2.4.x_en.zip
#!unzip -q milvus_docs_2.4.x_en.zip -d milvus_docs

我们从 `milvus_docs/en/faq` 文件夹加载所有 markdown 文件。对于每个文档，我们简单地使用 "# " 来分割文件中的内容，这样可以大致分离出 markdown 文件中每个主要部分的内容。

In [3]:
from glob import glob

text_lines = []

for file_path in glob("mfd.md", recursive=True):
    with open(file_path, "r") as file:
        file_text = file.read()

    text_lines += file_text.split("# ")

In [4]:
len(text_lines)

30

### 准备 LLM 和 Embedding 模型

DeepSeek 支持 OpenAI 风格的 API，您可以使用相同的 API 进行微小调整来调用 LLM。

In [5]:
from openai import OpenAI

deepseek_client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com/v1",  # DeepSeek API 的基地址
)

定义一个 embedding 模型，使用 `milvus_model` 来生成文本嵌入。我们以 `DefaultEmbeddingFunction` 模型为例，这是一个预训练的轻量级嵌入模型。

In [6]:
# from pymilvus import model as milvus_model

# embedding_model = milvus_model.DefaultEmbeddingFunction()

from pymilvus import model as milvus_model

# OpenAI国内代理 https://api.apiyi.com/token 
embedding_model = milvus_model.dense.OpenAIEmbeddingFunction(
    model_name='text-embedding-3-large', # Specify the model name
    api_key='sk-Rh0LNm9vr7ISKCBc635cEbD3826c4fC68bCcEe8521C30dAb', # Provide your OpenAI API key
    base_url='https://api.apiyi.com/v1',
    dimensions=512
)

生成一个测试嵌入并打印其维度和前几个元素。

In [7]:
test_embedding = embedding_model.encode_queries(["This is a test"])[0]
embedding_dim = len(test_embedding)
print(embedding_dim)
print(test_embedding[:10])

512
[-0.02814663  0.00428726 -0.01852599  0.08190062 -0.03156214 -0.05275258
 -0.04885425  0.12481797 -0.0208328   0.03966279]


In [8]:
test_embedding_0 = embedding_model.encode_queries(["That is a test"])[0]
print(test_embedding_0[:10])

[-0.00578664  0.02242682 -0.01892621  0.12811586 -0.01249751 -0.07321841
 -0.00281971  0.08617394 -0.04377401  0.03073668]


## 将数据加载到 Milvus

### 创建 Collection

In [10]:
from pymilvus import MilvusClient

milvus_client = MilvusClient(uri="./milvus_demo.db")

collection_name = "mfd_collection"

关于 `MilvusClient` 的参数：

*   将 `uri` 设置为本地文件，例如 `./milvus.db`，是最方便的方法，因为它会自动利用 Milvus Lite 将所有数据存储在此文件中。
*   如果您有大规模数据，可以在 Docker 或 Kubernetes 上设置性能更高的 Milvus 服务器。在此设置中，请使用服务器 URI，例如 `http://localhost:19530`，作为您的 `uri`。
*   如果您想使用 Zilliz Cloud（Milvus 的完全托管云服务），请调整 `uri` 和 `token`，它们对应 Zilliz Cloud 中的 Public Endpoint 和 Api key。

检查 collection 是否已存在，如果存在则删除它。

In [11]:
if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)

创建一个具有指定参数的新 collection。

如果我们不指定任何字段信息，Milvus 将自动创建一个默认的 `id` 字段作为主键，以及一个 `vector` 字段来存储向量数据。一个保留的 JSON 字段用于存储非 schema 定义的字段及其值。

`metric_type` (距离度量类型):
     作用：定义如何计算向量之间的相似程度。
     例如：`IP` (内积) - 值越大通常越相似；`L2` (欧氏距离) - 值越小越相似；`COSINE` (余弦相似度) - 通常转换为距离，值越小越相似。
     选择依据：根据你的嵌入模型的特性和期望的相似性定义来选择。

 `consistency_level` (一致性级别):
     作用：定义数据写入后，读取操作能多快看到这些新数据。
     例如：
         `Strong` (强一致性): 总是读到最新数据，可能稍慢。
         `Bounded` (有界过期): 可能读到几秒内旧数据，性能较好 (默认)。
         `Session` (会话一致性): 自己写入的自己能立刻读到。
         `Eventually` (最终一致性): 最终会读到新数据，但没时间保证，性能最好。
     选择依据：在数据实时性要求和系统性能之间做权衡。

简单来说：
 `metric_type`：怎么算相似。
 `consistency_level`：新数据多久能被读到。

In [11]:
milvus_client.create_collection(
    collection_name=collection_name,
    dimension=embedding_dim,
    metric_type="IP",  # 内积距离
    consistency_level="Strong",  # 支持的值为 (`"Strong"`, `"Session"`, `"Bounded"`, `"Eventually"`)。更多详情请参见 https://milvus.io/docs/consistency.md#Consistency-Level。
)

### 插入数据

遍历文本行，创建嵌入，然后将数据插入 Milvus。

这里有一个新字段 `text`，它是在 collection schema 中未定义的字段。它将自动添加到保留的 JSON 动态字段中，该字段在高级别上可以被视为普通字段。

In [12]:
from tqdm import tqdm

data = []

doc_embeddings = embedding_model.encode_documents(text_lines)

for i, line in enumerate(tqdm(text_lines, desc="Creating embeddings")):
    data.append({"id": i, "vector": doc_embeddings[i], "text": line})

milvus_client.insert(collection_name=collection_name, data=data)

Creating embeddings: 100%|██████████| 30/30 [00:00<00:00, 218833.25it/s]


{'insert_count': 30, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29], 'cost': 0}

## 构建 RAG

### 检索查询数据

我们指定一个关于 Milvus 的常见问题。

In [13]:
question = "合同金额如果变动怎么处理?"

在 collection 中搜索该问题，并检索语义上最匹配的前3个结果。

In [14]:
search_res = milvus_client.search(
    collection_name=collection_name,
    data=embedding_model.encode_queries(
        [question]
    ),  # 将问题转换为嵌入向量
    limit=3,  # 返回前3个结果
    search_params={"metric_type": "IP", "params": {}},  # 内积距离
    output_fields=["text"],  # 返回 text 字段
)

让我们看一下查询的搜索结果

In [15]:
import json

retrieved_lines_with_distances = [
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
]
print(json.dumps(retrieved_lines_with_distances, indent=4))

[
    [
        "\u7b2c\u4e09\u7ae0 \u5408\u540c\u7684\u53d8\u66f4\u548c\u8f6c\u8ba9\n\n**\u7b2c\u4e94\u767e\u56db\u5341\u516b\u6761** \u5f53\u4e8b\u4eba\u534f\u5546\u4e00\u81f4\uff0c\u53ef\u4ee5\u53d8\u66f4\u5408\u540c\u3002\n\n**\u7b2c\u4e94\u767e\u56db\u5341\u4e5d\u6761** \u5f53\u4e8b\u4eba\u5bf9\u5408\u540c\u53d8\u66f4\u7684\u5185\u5bb9\u7ea6\u5b9a\u4e0d\u660e\u786e\u7684\uff0c\u63a8\u5b9a\u4e3a\u672a\u53d8\u66f4\u3002\n\n**\u7b2c\u4e94\u767e\u4e94\u5341\u6761** \u503a\u6743\u4eba\u53ef\u4ee5\u5c06\u5408\u540c\u7684\u6743\u5229\u5168\u90e8\u6216\u8005\u90e8\u5206\u8f6c\u8ba9\u7ed9\u7b2c\u4e09\u4eba\uff0c\u4f46\u662f\u6709\u4e0b\u5217\u60c5\u5f62\u4e4b\u4e00\u7684\u9664\u5916\uff1a\n\uff08\u4e00\uff09\u6839\u636e\u5408\u540c\u6027\u8d28\u4e0d\u5f97\u8f6c\u8ba9\uff1b\n\uff08\u4e8c\uff09\u6309\u7167\u5f53\u4e8b\u4eba\u7ea6\u5b9a\u4e0d\u5f97\u8f6c\u8ba9\uff1b\n\uff08\u4e09\uff09\u4f9d\u7167\u6cd5\u5f8b\u89c4\u5b9a\u4e0d\u5f97\u8f6c\u8ba9\u3002\n\u503a\u6743\u4eba\u8f6c\u8ba9\u6743\u522

### 使用 LLM 获取 RAG 响应

将检索到的文档转换为字符串格式。

In [16]:
context = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances]
)

In [17]:
context

'第三章 合同的变更和转让\n\n**第五百四十八条** 当事人协商一致，可以变更合同。\n\n**第五百四十九条** 当事人对合同变更的内容约定不明确的，推定为未变更。\n\n**第五百五十条** 债权人可以将合同的权利全部或者部分转让给第三人，但是有下列情形之一的除外：\n（一）根据合同性质不得转让；\n（二）按照当事人约定不得转让；\n（三）依照法律规定不得转让。\n债权人转让权利的，应当通知债务人。未经通知，该转让对债务人不发生效力。\n\n**第五百五十一条** 债权人转让权利的，受让人取得与债权有关的从权利，但是该从权利专属于债权人自身的除外。\n\n**第五百五十二条** 债务人将合同的义务全部或者部分转让给第三人的，应当经债权人同意。\n\n**第五百五十三条** 债务人转让义务的，新债务人可以主张原债务人对债权人的抗辩。\n新债务人承担债务的，应当承担与主债务有关的从债务，但是该从债务专属于原债务人自身的除外。\n\n**第五百五十四条** 当事人一方经对方同意，可以将自己在合同中的权利和义务一并转让给第三人。\n\n**第五百五十五条** 权利和义务一并转让的，适用债权转让、债务转让的有关规定。\n\n**第五百五十六条** 合同变更的，不影响当事人请求损害赔偿的权利。\n\n###\n第一章 一般规定\n\n**第四百八十一条** 为了保护合同当事人的合法权益，维护社会经济秩序，促进社会主义现代化建设，制定本编。\n\n**第四百八十二条** 本编调整因合同产生的民事关系。\n\n**第四百八十三条** 合同是民事主体之间设立、变更、终止民事法律关系的协议。\n\n**第四百八十四条** 当事人订立合同，可以采用书面形式、口头形式或者其他形式。\n法律、行政法规规定采用书面形式的，应当采用书面形式。当事人约定采用书面形式的，应当采用书面形式。\n\n**第四百八十五条** 当事人可以参照各类合同的示范文本订立合同。\n\n**第四百八十六条** 合同内容由当事人约定，一般包括下列条款：\n（一）当事人的姓名或者名称和住所；\n（二）标的；\n（三）数量；\n（四）质量；\n（五）价款或者报酬；\n（六）履行期限、地点和方式；\n（七）违约责任；\n（八）解决争议的方法。\n\n**第四百八十七条** 合同的订立包括要约和承诺阶段。\n\n**第

In [18]:
question

'合同金额如果变动怎么处理?'

为语言模型定义系统和用户提示。此提示是使用从 Milvus 检索到的文档组装而成的。

In [19]:
SYSTEM_PROMPT = """
Human: 你是一个 AI 助手。你能够从提供的上下文段落片段中找到问题的答案。
"""
USER_PROMPT = f"""
请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。最后追加你对这个法条的理解，并写在我的理解：后面。
<context>
{context}
</context>
<question>
{question}
</question>
<translated>
</translated>
"""

In [20]:
USER_PROMPT

'\n请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。最后追加你对这个法条的理解，并写在我的理解：后面。\n<context>\n第三章 合同的变更和转让\n\n**第五百四十八条** 当事人协商一致，可以变更合同。\n\n**第五百四十九条** 当事人对合同变更的内容约定不明确的，推定为未变更。\n\n**第五百五十条** 债权人可以将合同的权利全部或者部分转让给第三人，但是有下列情形之一的除外：\n（一）根据合同性质不得转让；\n（二）按照当事人约定不得转让；\n（三）依照法律规定不得转让。\n债权人转让权利的，应当通知债务人。未经通知，该转让对债务人不发生效力。\n\n**第五百五十一条** 债权人转让权利的，受让人取得与债权有关的从权利，但是该从权利专属于债权人自身的除外。\n\n**第五百五十二条** 债务人将合同的义务全部或者部分转让给第三人的，应当经债权人同意。\n\n**第五百五十三条** 债务人转让义务的，新债务人可以主张原债务人对债权人的抗辩。\n新债务人承担债务的，应当承担与主债务有关的从债务，但是该从债务专属于原债务人自身的除外。\n\n**第五百五十四条** 当事人一方经对方同意，可以将自己在合同中的权利和义务一并转让给第三人。\n\n**第五百五十五条** 权利和义务一并转让的，适用债权转让、债务转让的有关规定。\n\n**第五百五十六条** 合同变更的，不影响当事人请求损害赔偿的权利。\n\n###\n第一章 一般规定\n\n**第四百八十一条** 为了保护合同当事人的合法权益，维护社会经济秩序，促进社会主义现代化建设，制定本编。\n\n**第四百八十二条** 本编调整因合同产生的民事关系。\n\n**第四百八十三条** 合同是民事主体之间设立、变更、终止民事法律关系的协议。\n\n**第四百八十四条** 当事人订立合同，可以采用书面形式、口头形式或者其他形式。\n法律、行政法规规定采用书面形式的，应当采用书面形式。当事人约定采用书面形式的，应当采用书面形式。\n\n**第四百八十五条** 当事人可以参照各类合同的示范文本订立合同。\n\n**第四百八十六条** 合同内容由当事人约定，一般包括下列条款：\n（一）当事人的姓名或者名称和住所；\n（二）标的；\n（三）数量；\n（四）

使用 DeepSeek 提供的 `deepseek-chat` 模型根据提示生成响应。

In [21]:
response = deepseek_client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT},
    ],
)
print(response.choices[0].message.content)

根据提供的上下文，合同金额的变动属于合同内容的变更，应遵循以下规定处理：

1. **当事人协商一致**：根据第五百四十八条，合同金额的变动需要当事人协商一致才能变更合同。
2. **变更内容明确**：根据第五百四十九条，如果对合同金额变动的约定不明确，则推定为未变更，即原合同金额仍然有效。
3. **不影响损害赔偿权利**：根据第五百五十六条，即使合同金额发生变更，当事人仍有权请求因变更可能造成的损害赔偿。

**我的理解**：合同金额的变动本质上属于合同变更，必须由双方明确协商并达成一致，且变更内容需清晰具体，否则视为未变更。这一规定旨在维护合同的稳定性和可预见性，同时保障当事人在变更过程中的合法权益，避免因模糊约定引发纠纷。如果变更导致损失，受损方仍可主张赔偿，体现了公平原则。


In [23]:
question = "居住权有有效期吗?"
search_res = milvus_client.search(
    collection_name=collection_name,
    data=embedding_model.encode_queries(
        [question]
    ),  # 将问题转换为嵌入向量
    limit=3,  # 返回前3个结果
    search_params={"metric_type": "IP", "params": {}},  # 内积距离
    output_fields=["text"],  # 返回 text 字段
)

In [24]:
import json

retrieved_lines_with_distances = [
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
]
print(json.dumps(retrieved_lines_with_distances, indent=4))

[
    [
        "\u7b2c\u4e94\u8282 \u5c45\u4f4f\u6743\n\n**\u7b2c\u4e09\u767e\u516b\u5341\u4e8c\u6761** \u5c45\u4f4f\u6743\u4eba\u6709\u6743\u4f9d\u7167\u5408\u540c\u7ea6\u5b9a\u6216\u8005\u9057\u5631\uff0c\u5bf9\u4ed6\u4eba\u7684\u4f4f\u5b85\u4eab\u6709\u5360\u6709\u3001\u4f7f\u7528\u7684\u6743\u5229\uff0c\u4ee5\u6ee1\u8db3\u751f\u6d3b\u5c45\u4f4f\u7684\u9700\u8981\u3002\n\n**\u7b2c\u4e09\u767e\u516b\u5341\u4e09\u6761** \u8bbe\u7acb\u5c45\u4f4f\u6743\uff0c\u5f53\u4e8b\u4eba\u5e94\u5f53\u4f9d\u7167\u6cd5\u5f8b\u89c4\u5b9a\u529e\u7406\u767b\u8bb0\u3002\n\u672a\u7ecf\u767b\u8bb0\uff0c\u4e0d\u53d1\u751f\u7269\u6743\u6548\u529b\u3002\n\n**\u7b2c\u4e09\u767e\u516b\u5341\u56db\u6761** \u5c45\u4f4f\u6743\u4e0d\u5f97\u8f6c\u8ba9\u3001\u7ee7\u627f\u3002\n\u8bbe\u7acb\u5c45\u4f4f\u6743\u7684\u4f4f\u5b85\u4e0d\u5f97\u51fa\u79df\uff0c\u4f46\u662f\u5f53\u4e8b\u4eba\u53e6\u6709\u7ea6\u5b9a\u7684\u9664\u5916\u3002\n\n**\u7b2c\u4e09\u767e\u516b\u5341\u4e94\u6761** \u5c45\u4f4f\u6743\u671f\u95f4\u5c4a

In [25]:
context = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances]
)

In [26]:
SYSTEM_PROMPT = """
Human: 你是一个 AI 助手。你能够从提供的上下文段落片段中找到问题的答案。
"""
USER_PROMPT = f"""
请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。最后追加你对这个法条的理解，并写在我的理解：后面。
<context>
{context}
</context>
<question>
{question}
</question>
<translated>
</translated>
"""

In [27]:
response = deepseek_client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT},
    ],
)
print(response.choices[0].message.content)

根据提供的上下文，居住权是有有效期的。具体依据是《中华人民共和国民法典》第三百八十五条，该条规定：“居住权期间届满或者居住权人死亡的，居住权消灭。” 这表明居住权的存续可以基于一个明确的期间，当该期间届满时，居住权即告消灭。

我的理解：居住权是一种用益物权，其设立目的主要是为了保障特定人的生活居住需要，因此具有人身专属性。法条明确规定居住权不得转让、继承，这进一步强化了其与居住权人个人紧密关联的特性。关于有效期，法律允许通过合同或遗嘱设定一个具体的居住权期间。一旦期间届满，或者居住权人死亡（这被视为一种法定的权利终止事件），居住权便自动消灭，之后需要办理注销登记。这种设计平衡了住宅所有权人的物权利益和居住权人的居住保障需求，使得居住权成为一种有期限的、稳定的居住保障，而非永久性的权利。
